In [15]:
#!/usr/bin/env python3
"""
PCA + HMM Multi-Horizon Trading Signal Generator
Complete implementation for cryptocurrency trading with PCA + Hidden Markov Model

This follows the structure of Park (2023),
"Principal Component Analysis and Hidden Markov Model for Forecasting Stock Returns":
- PCA on (normalized) asset return matrix
- Keep enough PCs to explain (1 - noise_pct) of variance
- Gaussian HMM on factor returns
- Forecast next-step factor means using transition matrix
- Map back to asset space via eigenvectors

Key features:
- One PCA+HMM model per horizon (1, 3, 6 bars)
- Cross-asset factor modeling
- Regime-based state transitions
- Walk-forward cross-validation
- Isotonic + conformal calibration
- JSON feed output for trading agent
"""

from __future__ import annotations

import json
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Iterable, Sequence

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import (
    confusion_matrix, classification_report,
    precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, precision_recall_curve,
    average_precision_score
)

import matplotlib
matplotlib.use('Agg')  # Non-interactive backend
import matplotlib.pyplot as plt
import seaborn as sns

from hmmlearn.hmm import GaussianHMM  # pip install hmmlearn
from scipy.stats import norm

import warnings
warnings.filterwarnings("ignore")

In [16]:
# =========================================
# CONFIGURATION
# =========================================
DATA_DIR = Path("/Users/yuhaoli/code/MAS_For_Finance/ML_Final_Project/Bybit_CSV_Data")
FILES = {
    "BTC": DATA_DIR / "Bybit_BTC.csv",
    "ETH": DATA_DIR / "Bybit_ETH.csv",
    "SOL": DATA_DIR / "Bybit_SOL.csv",
    "XRP": DATA_DIR / "Bybit_XRP.csv",
    "DOGE": DATA_DIR / "Bybit_DOGE.csv",
}

HORIZONS = [1, 3, 6]  # Forecast horizons in 4-hour bars
DEFAULT_COST_BP = {1: 8.0, 3: 10.0, 6: 12.0}  # Trading costs in basis points

# PCA+HMM hyperparameters
NOISE_PCT = 0.30          # Fraction of variance to discard as noise
N_STATES_GRID = (2, 3, 4, 5, 6, 7, 8)  # Candidate HMM states

# Policy thresholds
TAU_P = 0.60
TAU_MU = 0.0005
LAM = 2.0
W_MAX = 0.50

MODEL_VERSION = "pca_hmm_multiH_v1.0"
CALIBRATION_VERSION = "iso+conformal_v1"

In [17]:
# =========================================
# UTILITY FUNCTIONS
# =========================================
def bp_to_logret(bp: float) -> float:
    """Convert basis points to log-return units."""
    return bp * 1e-4


def _find_close_column(df: pd.DataFrame) -> str:
    """Find the close price column in a dataframe."""
    lower = {c.lower(): c for c in df.columns}
    for key in ("close", "closing_price", "close_price", "price_close", "last", "c"):
        if key in lower:
            return lower[key]
    float_cols = [c for c in df.columns if pd.api.types.is_float_dtype(df[c])]
    if len(float_cols) == 1:
        return float_cols[0]
    raise ValueError("Cannot identify 'close' column.")


def cumulative_log_returns(price: pd.Series, h: int) -> pd.Series:
    """Compute log(P_{t+h}/P_t) aligned to t."""
    return np.log(price.shift(-h) / price).dropna()


def brier_score(y: np.ndarray, p: np.ndarray) -> float:
    """Brier score for probability calibration."""
    return float(np.mean((y - p) ** 2))


def expected_calibration_error(y: np.ndarray, p: np.ndarray, bins: int = 10) -> float:
    """Expected Calibration Error (ECE)."""
    edges = np.linspace(0, 1, bins + 1)
    ece = 0.0
    for i in range(bins):
        m = (p >= edges[i]) & (p < edges[i+1])
        if m.sum() == 0:
            continue
        ece += (m.sum()/len(p)) * np.abs(np.mean(y[m]) - np.mean(p[m]))
    return float(ece)

In [18]:
# =========================================
# EVALUATION METRICS
# =========================================
def compute_classification_metrics(y_true: np.ndarray, y_pred: np.ndarray,
                                   y_prob: np.ndarray = None) -> dict:
    """Compute comprehensive classification metrics."""
    metrics = {}
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    metrics['confusion_matrix'] = cm
    metrics['true_negatives'] = int(tn)
    metrics['false_positives'] = int(fp)
    metrics['false_negatives'] = int(fn)
    metrics['true_positives'] = int(tp)
    metrics['accuracy'] = float((tp + tn) / (tp + tn + fp + fn))
    metrics['precision'] = precision_score(y_true, y_pred, zero_division=0)
    metrics['recall'] = recall_score(y_true, y_pred, zero_division=0)
    metrics['f1_score'] = f1_score(y_true, y_pred, zero_division=0)
    metrics['specificity'] = float(tn / (tn + fp)) if (tn + fp) > 0 else 0.0
    metrics['balanced_accuracy'] = (metrics['recall'] + metrics['specificity']) / 2
    if y_prob is not None and len(np.unique(y_true)) > 1:
        try:
            metrics['roc_auc'] = roc_auc_score(y_true, y_prob)
            metrics['average_precision'] = average_precision_score(y_true, y_prob)
        except:
            metrics['roc_auc'] = None
            metrics['average_precision'] = None
    return metrics


def print_classification_report(metrics: dict, horizon: int, split: str = "test"):
    """Pretty print classification metrics."""
    print(f"\n{'='*60}")
    print(f"CLASSIFICATION METRICS - Horizon {horizon} ({split})")
    print(f"{'='*60}")
    cm = metrics['confusion_matrix']
    print("\nConfusion Matrix:")
    print(f"                Predicted Negative  Predicted Positive")
    print(f"Actual Negative        {cm[0,0]:6d}              {cm[0,1]:6d}")
    print(f"Actual Positive        {cm[1,0]:6d}              {cm[1,1]:6d}")
    print(f"\nPerformance Metrics:")
    print(f"  Accuracy:           {metrics['accuracy']:.4f}")
    print(f"  Balanced Accuracy:  {metrics['balanced_accuracy']:.4f}")
    print(f"  Precision:          {metrics['precision']:.4f}")
    print(f"  Recall (TPR):       {metrics['recall']:.4f}")
    print(f"  Specificity (TNR):  {metrics['specificity']:.4f}")
    print(f"  F1 Score:           {metrics['f1_score']:.4f}")
    if metrics.get('roc_auc') is not None:
        print(f"  ROC-AUC:            {metrics['roc_auc']:.4f}")


def save_metrics_to_csv(all_metrics: dict, save_path: Path):
    """Save metrics summary to CSV."""
    rows = []
    for symbol, horizons in all_metrics.items():
        for h, metrics in horizons.items():
            row = {
                'symbol': symbol, 'horizon': h,
                'accuracy': metrics.get('accuracy', np.nan),
                'balanced_accuracy': metrics.get('balanced_accuracy', np.nan),
                'precision': metrics.get('precision', np.nan),
                'recall': metrics.get('recall', np.nan),
                'f1_score': metrics.get('f1_score', np.nan),
                'roc_auc': metrics.get('roc_auc', np.nan),
                'brier_score': metrics.get('brier_val', np.nan),
                'ece': metrics.get('ece_val', np.nan),
            }
            rows.append(row)
    df = pd.DataFrame(rows)
    df.to_csv(save_path, index=False)
    print(f"\nSaved metrics to: {save_path}")

In [19]:
# =========================================
# FEATURE ENGINEERING
# =========================================
def make_feature_table(close: pd.Series):
    """Build feature table from close prices."""
    df = pd.DataFrame(index=close.index)
    df["price"] = close.astype(float)
    df["ret_1"] = np.log(df["price"] / df["price"].shift(1))
    df["ret_3"] = np.log(df["price"] / df["price"].shift(3))
    df["ret_6"] = np.log(df["price"] / df["price"].shift(6))
    df["vol_6"] = df["ret_1"].rolling(6).std()
    df["vol_12"] = df["ret_1"].rolling(12).std()
    ma_10 = df["price"].rolling(10).mean()
    ma_20 = df["price"].rolling(20).mean()
    df["ma_ratio"] = np.log(ma_10 / ma_20)
    df = df.dropna()
    feat_cols = [c for c in df.columns if c != "price"]
    X = df[feat_cols].values.astype(float)
    return df, X

In [20]:
# =========================================
# WALK-FORWARD CV
# =========================================
def purged_walkforward_slices(n: int, n_folds: int = 3, embargo: int = 24):
    """Generate (train, val, test) slices for walk-forward CV."""
    fold_size = n // (n_folds + 2)
    slices = []
    for i in range(n_folds):
        train_end = (i + 1) * fold_size
        val_start = train_end + embargo
        val_end = val_start + fold_size
        test_start = val_end + embargo
        test_end = min(test_start + fold_size, n)
        if test_end - test_start < fold_size // 2:
            break
        slices.append(((0, train_end), (val_start, val_end), (test_start, test_end)))
    return slices

In [21]:
# =========================================
# PCA + HMM MODEL
# =========================================
@dataclass
class PCAHMMSnapshot:
    """Container for trained PCA+HMM model."""
    hmm: GaussianHMM
    scaler: StandardScaler
    eigenvectors: np.ndarray
    mu_X: np.ndarray
    sigma_X: np.ndarray
    horizon: int
    n_factors: int
    n_states: int
    feature_names: list[str] | None = None


def fit_pca_hmm(X_train: np.ndarray, y_train: np.ndarray, horizon: int,
                feature_names: list[str] = None, noise_pct: float = NOISE_PCT,
                n_states_grid: Iterable[int] = N_STATES_GRID,
                random_state: int = 123) -> PCAHMMSnapshot:
    """Fit PCA + Gaussian HMM on feature matrix."""
    T, D = X_train.shape
    scaler = StandardScaler().fit(X_train)
    X_scaled = scaler.transform(X_train)
    mu_X = X_scaled.mean(axis=0)
    sigma_X = X_scaled.std(axis=0, ddof=1)
    sigma_X[sigma_X == 0.0] = 1e-8
    Y = (X_scaled - mu_X) / sigma_X
    H = np.cov(Y, rowvar=False)
    eigvals, eigvecs = np.linalg.eigh(H)
    idx = np.argsort(eigvals)[::-1]
    eigvals, eigvecs = eigvals[idx], eigvecs[:, idx]
    total_var = eigvals.sum()
    if total_var <= 0:
        k = 1
    else:
        cum_ratio = np.cumsum(eigvals) / total_var
        k = int(np.searchsorted(cum_ratio, 1.0 - noise_pct) + 1)
        k = max(1, min(k, D))
    E_k = eigvecs[:, :k]
    F = Y @ E_k
    best_model, best_aic, best_n_states = None, np.inf, 2
    for n_states in n_states_grid:
        try:
            hmm = GaussianHMM(n_components=n_states, covariance_type="diag",
                              n_iter=200, random_state=random_state)
            hmm.fit(F)
            logL = hmm.score(F)
            n_params = (n_states - 1) + n_states * (n_states - 1) + 2 * n_states * k
            aic = -2.0 * logL + 2.0 * n_params
            if aic < best_aic:
                best_aic, best_model, best_n_states = aic, hmm, n_states
        except:
            continue
    if best_model is None:
        best_model = GaussianHMM(n_components=2, covariance_type="diag",
                                  n_iter=200, random_state=random_state)
        best_model.fit(F)
        best_n_states = 2
    return PCAHMMSnapshot(hmm=best_model, scaler=scaler, eigenvectors=E_k,
                          mu_X=mu_X, sigma_X=sigma_X, horizon=horizon,
                          n_factors=k, n_states=best_n_states, feature_names=feature_names)

In [22]:
# =========================================
# FORECASTING
# =========================================
def forecast_multi_horizon_pca_hmm(snapshots: dict[int, PCAHMMSnapshot],
                                    X_seg: np.ndarray, price_seg: pd.Series,
                                    horizons: list[int], cost_bp: dict = None):
    """Generate multi-horizon forecasts using trained PCA+HMM models."""
    if cost_bp is None:
        cost_bp = {h: DEFAULT_COST_BP.get(h, 8.0) for h in horizons}
    cost_log = {h: bp_to_logret(float(cost_bp[h])) for h in horizons}
    Tseg, idx, out = X_seg.shape[0], price_seg.index, {}
    z10, z90 = norm.ppf(0.10), norm.ppf(0.90)
    for h in horizons:
        if h not in snapshots:
            continue
        snap = snapshots[h]
        T_h = Tseg - h
        if T_h <= 0:
            out[h] = pd.DataFrame(index=idx[:0])
            continue
        X_scaled = snap.scaler.transform(X_seg[:T_h])
        Y = (X_scaled - snap.mu_X) / snap.sigma_X
        F = Y @ snap.eigenvectors
        hmm = snap.hmm
        posteriors = hmm.predict_proba(F)
        trans, means_f, covars_f = hmm.transmat_, hmm.means_, hmm.covars_
        N, k = means_f.shape
        E_k = snap.eigenvectors  # shape (D, k)
        D = E_k.shape[0]
        E_sq = E_k ** 2  # shape (D, k)

        # Map factor means back to feature space
        feat_mean_state_std = means_f @ E_k.T  # (N, D)
        feat_mean_state = feat_mean_state_std * snap.sigma_X + snap.mu_X  # (N, D)

        # Handle covariance - extract diagonal variances properly
        # covars_f can be (N, k) for diag or (N, k, k) for full
        if covars_f.ndim == 2:
            # Shape (N, k) - diagonal variances in factor space
            factor_vars = covars_f
        elif covars_f.ndim == 3:
            # Shape (N, k, k) - extract diagonal
            factor_vars = np.array([np.diag(covars_f[j]) for j in range(N)])
        else:
            factor_vars = covars_f.reshape(N, -1)

        # Compute feature variances: var(y_i) = sum_m E_{i,m}^2 * var(f_m)
        # For each state j: (D,) = sum over k of (D,k) * (k,) -> need to broadcast correctly
        feat_var_state = np.zeros((N, D))
        for j in range(N):
            # factor_vars[j] has shape (k,), E_sq has shape (D, k)
            # We want: for each feature i, sum_m E_sq[i,m] * factor_vars[j,m]
            feat_var_state[j, :] = (E_sq * factor_vars[j]).sum(axis=1) * (snap.sigma_X ** 2)

        eps, mu_pred, std_pred, p_edge = 1e-12, np.zeros(T_h), np.zeros(T_h), np.zeros(T_h)
        feat_idx = 0  # Use first feature (ret_1) as prediction target

        for t in range(T_h):
            alpha = hmm.startprob_ if t == 0 else posteriors[t - 1]
            beta = alpha @ trans
            mix_mean = beta @ feat_mean_state[:, feat_idx]
            mix_second = beta @ (feat_var_state[:, feat_idx] + feat_mean_state[:, feat_idx] ** 2)
            mix_var = np.maximum(mix_second - mix_mean ** 2, eps)
            mix_std = np.sqrt(mix_var)
            p_edge_val = sum(beta[j] * (1.0 - norm.cdf((cost_log[h] - feat_mean_state[j, feat_idx]) /
                             np.sqrt(np.maximum(feat_var_state[j, feat_idx], eps)))) for j in range(N))
            mu_pred[t], std_pred[t], p_edge[t] = mix_mean, mix_std, p_edge_val

        p_now = price_seg.iloc[:T_h].values
        out_h = pd.DataFrame(index=idx[:T_h])
        out_h['mu'], out_h['std'], out_h['p_edge_raw'] = mu_pred, std_pred, p_edge
        out_h['ret_q10'], out_h['ret_q50'], out_h['ret_q90'] = mu_pred + z10 * std_pred, mu_pred, mu_pred + z90 * std_pred
        out_h['price_pred'] = p_now * np.exp(mu_pred)
        out_h['price_q10'], out_h['price_q50'], out_h['price_q90'] = p_now * np.exp(out_h['ret_q10']), p_now * np.exp(mu_pred), p_now * np.exp(out_h['ret_q90'])
        out[h] = out_h
    return out, cost_log

In [23]:
# =========================================
# CALIBRATION
# =========================================
@dataclass
class ProbCalibrator:
    method: str
    iso: IsotonicRegression | None = None

def fit_prob_calibrator_isotonic(p_raw: np.ndarray, y: np.ndarray, min_points: int = 30) -> ProbCalibrator:
    p_raw, y = np.asarray(p_raw, float), np.asarray(y, float)
    m = np.isfinite(p_raw) & np.isfinite(y)
    p, t = p_raw[m], y[m]
    if p.size < min_points or np.unique(p).size < 3:
        return ProbCalibrator(method="identity")
    iso = IsotonicRegression(out_of_bounds="clip")
    iso.fit(p, t)
    return ProbCalibrator(method="isotonic", iso=iso)

def apply_prob_calibrator(cal: ProbCalibrator, p_raw: np.ndarray) -> np.ndarray:
    p_raw = np.asarray(p_raw, float)
    return cal.iso.predict(p_raw) if cal.method == "isotonic" else p_raw

@dataclass
class IntervalCalibrator:
    method: str
    q_alpha: float
    alpha: float

def fit_conformal_interval(residuals: np.ndarray, alpha: float = 0.2) -> IntervalCalibrator:
    resid = np.asarray(residuals, float)
    resid = resid[np.isfinite(resid)]
    q = float(np.quantile(np.abs(resid), 1 - alpha)) if resid.size > 0 else 0.0
    return IntervalCalibrator(method="conformal_abs", q_alpha=q, alpha=alpha)

def apply_conformal_interval(cal: IntervalCalibrator, mu: np.ndarray):
    mu = np.asarray(mu, float)
    return mu - cal.q_alpha, mu + cal.q_alpha

In [24]:
# =========================================
# MAIN TRAINING PIPELINE
# =========================================
def run_pca_hmm_for_symbol(symbol: str, path: Path, horizons: list[int] = HORIZONS,
                           n_folds: int = 3, embargo: int = 24):
    """Train and evaluate PCA+HMM models for one symbol."""
    df_raw = pd.read_csv(path)
    close_col = _find_close_column(df_raw)
    close = pd.Series(df_raw[close_col].astype(float).values, index=pd.RangeIndex(len(df_raw)))
    feat_df, X = make_feature_table(close)
    price, n = feat_df["price"], len(feat_df)
    feature_names = [c for c in feat_df.columns if c != "price"]
    folds = purged_walkforward_slices(n, n_folds=n_folds, embargo=embargo)
    results = {h: {"val": [], "test": [], "diag": [], "metrics": []} for h in horizons}
    print(f"\n{'='*60}\nTraining PCA+HMM for {symbol}\nHorizons: {horizons}, Folds: {n_folds}\n{'='*60}\n")
    for fold_idx, ((s0,e0), (s1,e1), (s2,e2)) in enumerate(folds):
        print(f"Fold {fold_idx + 1}/{len(folds)}: Train[{s0}:{e0}] Val[{s1}:{e1}] Test[{s2}:{e2}]")
        snapshots = {}
        for h in horizons:
            print(f"  Training h={h}...", end=" ")
            ret_train = cumulative_log_returns(price.iloc[s0:e0], h)
            n_train = min(len(X[s0:e0]), len(ret_train))
            X_train_aligned = X[s0:s0+n_train]
            if len(X_train_aligned) < 100:
                print("SKIP")
                continue
            y_train = (ret_train.iloc[:n_train].values > bp_to_logret(DEFAULT_COST_BP[h])).astype(int)
            try:
                snap = fit_pca_hmm(X_train_aligned, y_train, h, feature_names, random_state=123+fold_idx)
                snapshots[h] = snap
                print(f"OK (factors={snap.n_factors}, states={snap.n_states})")
            except Exception as e:
                print(f"ERROR: {e}")
        if not snapshots:
            continue
        print("  Forecasting...", end=" ")
        out_val_raw, cost_log = forecast_multi_horizon_pca_hmm(snapshots, X[s1:e1], price.iloc[s1:e1], horizons)
        out_test_raw, _ = forecast_multi_horizon_pca_hmm(snapshots, X[s2:e2], price.iloc[s2:e2], horizons)
        print("OK")
        for h in horizons:
            if h not in out_val_raw or h not in out_test_raw:
                continue
            ret_val = cumulative_log_returns(price.iloc[s1:e1], h)
            idx_common = out_val_raw[h].index.intersection(ret_val.index)
            if len(idx_common) < 20:
                continue
            dfV = out_val_raw[h].loc[idx_common].copy()
            maskV = np.isfinite(dfV["p_edge_raw"].values) & np.isfinite(dfV["mu"].values)
            dfV = dfV[maskV]
            if len(dfV) < 20:
                continue
            ret_val_aligned = ret_val.loc[dfV.index]
            y_val = (ret_val_aligned.values > cost_log[h]).astype(int)
            p_raw_val, mu_val = dfV["p_edge_raw"].values, dfV["mu"].values
            cal_prob = fit_prob_calibrator_isotonic(p_raw_val, y_val)
            resid_val = ret_val_aligned.values - mu_val
            cal_pi = fit_conformal_interval(resid_val)
            ret_test = cumulative_log_returns(price.iloc[s2:e2], h)
            idx_test_common = out_test_raw[h].index.intersection(ret_test.index)
            dfT = out_test_raw[h].loc[idx_test_common].copy()
            maskT = np.isfinite(dfT["p_edge_raw"].values) & np.isfinite(dfT["mu"].values)
            dfT = dfT[maskT]
            if len(dfT) == 0:
                continue
            dfT["p_edge"] = apply_prob_calibrator(cal_prob, dfT["p_edge_raw"].values)
            lo, hi = apply_conformal_interval(cal_pi, dfT["mu"].values)
            dfT["ret_lo"], dfT["ret_hi"] = lo, hi
            p_now = price.loc[dfT.index].values
            dfT["price_lo"], dfT["price_hi"] = p_now * np.exp(lo), p_now * np.exp(hi)
            dfT["edge"] = dfT["mu"] - cost_log[h]
            dfT["risk_edge"] = dfT["edge"] / (dfT["std"] + 1e-12)
            results[h]["test"].append(dfT)
            ret_test_aligned = ret_test.loc[dfT.index]
            y_test_true = (ret_test_aligned.values > cost_log[h]).astype(int)
            y_test_pred = (dfT["p_edge"].values > 0.5).astype(int)
            test_metrics = compute_classification_metrics(y_test_true, y_test_pred, dfT["p_edge"].values)
            print_classification_report(test_metrics, h)
            results[h]["metrics"].append(test_metrics)
            p_cal_val = apply_prob_calibrator(cal_prob, p_raw_val)
            brier = brier_score(y_val, p_cal_val)
            ece = expected_calibration_error(y_val, p_cal_val)
            coverage = float(np.mean((resid_val >= -cal_pi.q_alpha) & (resid_val <= cal_pi.q_alpha)))
            results[h]["diag"].append({"h": h, "brier_val": brier, "ece_val": ece, "pi_coverage_val": coverage})
    for h in horizons:
        for split in ("val", "test"):
            results[h][split] = pd.concat(results[h][split]).sort_index() if results[h][split] else pd.DataFrame()
    print(f"\nCompleted {symbol}\n")
    return results

In [25]:
# =========================================
# JSON EXPORT
# =========================================
def build_json_records(all_outputs: dict, model_version: str = MODEL_VERSION,
                       calibration_version: str = CALIBRATION_VERSION, horizons: list[int] = HORIZONS):
    """Build JSONL records for trading agent."""
    records = []
    for sym, res in all_outputs.items():
        for h in horizons:
            df = res[h]["test"]
            if isinstance(df, (list, tuple)):
                df = pd.concat(df).sort_index()
            for t, row in df.iterrows():
                rec = {
                    "timestamp_index": int(t), "symbol": sym, "horizon_bars": int(h),
                    "model_version": model_version, "calibration_version": calibration_version,
                    "signals": {
                        "expected_return": float(row["mu"]), "stdev_return": float(row["std"]),
                        "p_edge_gt_cost": float(row["p_edge"]), "predicted_price": float(row["price_pred"]),
                        "price_PI": {"p10": float(row["price_q10"]), "p50": float(row["price_q50"]), "p90": float(row["price_q90"])}
                    },
                    "policy_suggestions": {
                        "gate_threshold_p": TAU_P, "gate_threshold_edge": TAU_MU,
                        "suggested_action": "buy" if (row["p_edge"]>=TAU_P and row["edge"]>=TAU_MU and row["mu"]>0)
                                            else ("sell" if (row["p_edge"]>=TAU_P and row["edge"]>=TAU_MU and row["mu"]<0) else "hold")
                    }
                }
                records.append(rec)
    return records

In [26]:
# =========================================
# MAIN EXECUTION
# =========================================
if __name__ == "__main__":
    all_outputs, all_metrics = {}, {}
    for symbol, path in FILES.items():
        if not path.exists():
            print(f"Warning: {path} not found, skipping {symbol}")
            continue
        results = run_pca_hmm_for_symbol(symbol, path, HORIZONS, n_folds=3, embargo=24)
        all_outputs[symbol] = results
        all_metrics[symbol] = {}
        for h in HORIZONS:
            if results[h]['metrics']:
                metrics_list = results[h]['metrics']
                avg_metrics = {
                    'accuracy': np.mean([m['accuracy'] for m in metrics_list]),
                    'balanced_accuracy': np.mean([m['balanced_accuracy'] for m in metrics_list]),
                    'precision': np.mean([m['precision'] for m in metrics_list]),
                    'recall': np.mean([m['recall'] for m in metrics_list]),
                    'f1_score': np.mean([m['f1_score'] for m in metrics_list]),
                    'roc_auc': np.mean([m['roc_auc'] for m in metrics_list if m.get('roc_auc')]),
                }
                diag_list = results[h]['diag']
                if diag_list:
                    avg_metrics['brier_val'] = np.mean([d['brier_val'] for d in diag_list])
                    avg_metrics['ece_val'] = np.mean([d['ece_val'] for d in diag_list])
                all_metrics[symbol][h] = avg_metrics
        print(f"\n{'='*60}\nSUMMARY FOR {symbol}\n{'='*60}")
        for h in HORIZONS:
            if len(results[h]['test']) > 0 and h in all_metrics[symbol]:
                m = all_metrics[symbol][h]
                print(f"\nHorizon {h}: Acc={m['accuracy']:.4f}, F1={m['f1_score']:.4f}, AUC={m.get('roc_auc', 0):.4f}")
    metrics_csv = DATA_DIR / "pca_hmm_metrics_summary.csv"
    save_metrics_to_csv(all_metrics, metrics_csv)
    json_records = build_json_records(all_outputs)
    json_path = DATA_DIR / "trader_feed_pca_hmm_multiH.jsonl"
    with open(json_path, "w") as f:
        for r in json_records:
            f.write(json.dumps(r) + "\n")
    print(f"\n{'='*60}\nFINAL OUTPUTS\n{'='*60}")
    print(f"JSON feed:    {json_path}")
    print(f"Metrics CSV:  {metrics_csv}")
    print(f"{'='*60}\n")
    print("Tip: PCA+HMM captures market regimes and factor dynamics!")


Training PCA+HMM for BTC
Horizons: [1, 3, 6], Folds: 3

Fold 1/3: Train[0:1749] Val[1773:3522] Test[3546:5295]
  Training h=1... OK (factors=3, states=7)
  Training h=3... OK (factors=3, states=7)
  Training h=6... OK (factors=3, states=7)
  Forecasting... OK

CLASSIFICATION METRICS - Horizon 1 (test)

Confusion Matrix:
                Predicted Negative  Predicted Positive
Actual Negative           901                  52
Actual Positive           763                  32

Performance Metrics:
  Accuracy:           0.5338
  Balanced Accuracy:  0.4928
  Precision:          0.3810
  Recall (TPR):       0.0403
  Specificity (TNR):  0.9454
  F1 Score:           0.0728
  ROC-AUC:            0.4910

CLASSIFICATION METRICS - Horizon 3 (test)

Confusion Matrix:
                Predicted Negative  Predicted Positive
Actual Negative           921                  13
Actual Positive           809                   3

Performance Metrics:
  Accuracy:           0.5292
  Balanced Accuracy:  0.4949
